In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import re
from pandas.api.types import is_numeric_dtype

import requests
import IPython.display as Disp
import plotly.express as px
import sklearn
from IPython.core.display import display, HTML
from sklearn.decomposition import TruncatedSVD

/var/folders/7n/62525jfd52q__72l6v12gxfm0000gn/T/ipykernel_67982/1740148243.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
books = pd.read_feather("Finish/booksDB.feather")
books.drop(columns=["index"], inplace=True)

users = pd.read_feather("Finish/users_interaction.feather")
users.drop(columns=["index"], inplace=True)

print("Books Data:    ", books.shape)
print("Users Data:    ", users.shape)

In [4]:
#Analizamos la bd de Books
print("Columns: ", list(books.columns))
books.head()

Columns:  ['isbn', 'average_rating', 'description', 'authors', 'publisher', 'isbn13', 'publication_year', 'image_url', 'book_id', 'ratings_count', 'title', 'title_without_series']


,isbn,average_rating,description,authors,publisher,isbn13,publication_year,image_url,book_id,ratings_count,title,title_without_series
0,0312853122,4.00,,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,9780312853129,1984,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,3.23,"Anita Diamant's international bestseller ""The ...","[{'author_id': '626222', 'role': ''}]",Simon & Schuster Audio,9780743509985,2001,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10.0,Good Harbor,Good Harbor
2,,4.03,Omnibus book club edition containing the Ladie...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",,1987,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3.49,Addie Downs and Valerie Adler were eight when ...,"[{'author_id': '9212', 'role': ''}]",Atria Books,9780743294294,2009,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,Best Friends Forever,Best Friends Forever
4,0850308712,3.40,,"[{'author_id': '149918', 'role': ''}]",,9780850308716,100,https://images.gr-assets.com/books/1413219371m...,287140,15.0,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...


In [5]:
books.isnull().sum()

isbn                    0
average_rating          0
description             0
authors                 0
publisher               0
isbn13                  0
publication_year        0
image_url               0
book_id                 0
ratings_count           0
title                   0
title_without_series    0
dtype: int64

In [7]:
#Revisamos sintaxis de ISBN
flag = 0
k =[]
reg = "[^A-Za-z0-9]"

for x in books['isbn']:
    z = re.search(reg,x)    
    if z:
        flag = 1

if flag == 1:
    print("False")
else:
    print("True")

False


In [9]:
ratings_valid = users[users['rating'] != 0]
ratings_valid = ratings_valid.reset_index(drop = True)
ratings_valid.shape
ratings_valid.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [10]:
rating_count = pd.DataFrame(ratings_valid.groupby('book_id')['rating'].count())
rating_count.sort_values('rating', ascending=False).head()

,rating
book_id,
943,281135
536,273191
1000,230084
786,191018
941,172207


In [11]:
#Usar usuarios con 200 o más ratings y libros con 100 o más ratings
counts1 = ratings_valid['user_id'].value_counts()
ratings_valid = ratings_valid[ratings_valid['user_id'].isin(counts1[counts1 >= 200].index)]
counts = ratings_valid['rating'].value_counts()
ratings_valid = ratings_valid[ratings_valid['rating'].isin(counts[counts >= 100].index)]

In [13]:
books = books.astype({"book_id": "int32"})

In [14]:
#Juntar los df de books con este de ratings validos
combine_book_rating = pd.merge(ratings_valid, books, on='book_id')
combine_book_rating.head()

,user_id,book_id,is_read,rating,is_reviewed,isbn,average_rating,description,authors,publisher,isbn13,publication_year,image_url,ratings_count,title,title_without_series
0,0,947,1,5,1,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions
1,32,947,1,3,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions
2,224,947,1,4,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions
3,284,947,1,4,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions
4,311,947,1,2,1,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions


In [15]:
#Agrupamos por título y creamos un Rating Count Total
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
book_ratingCount.head()

,title,totalRatingCount
0,!Búscalo! (Look It Up!): A Quick Reference Gui...,3
1,!Exacto!: A Practical Guide to Spanish Grammar,3
2,!Fish! La Eficacia de un Equipo Radica en Su C...,73
3,""" Superman Returns "": The Visual Guide",3
4,"""...officer down, code three.""",2


In [16]:
#Combinamos con el total Rating count para saber los libros más populares
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,user_id,book_id,is_read,rating,is_reviewed,isbn,average_rating,description,authors,publisher,isbn13,publication_year,image_url,ratings_count,title,title_without_series,totalRatingCount
0,0,947,1,5,1,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions,5875
1,32,947,1,3,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions,5875
2,224,947,1,4,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions,5875
3,284,947,1,4,0,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions,5875
4,311,947,1,2,1,0373510055,3.73,Escaping the rat race and the lure of gold-dig...,"[{'author_id': '625', 'role': ''}]",,9780373510054,100,https://s.gr-assets.com/assets/nophoto/book/11...,27.0,First Impressions,First Impressions,5875


In [17]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   338484.000
mean        75.932
std        701.604
min          1.000
25%          2.000
50%          6.000
75%         24.000
max      81271.000
Name: totalRatingCount, dtype: float64


In [18]:
#Como el libro en la mediana solo se calificó pocas veces, chequemos la distibución superior
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.900     91.000
0.910    105.000
0.920    123.000
0.930    145.000
0.940    176.000
0.950    220.000
0.960    286.000
0.970    392.000
0.980    594.000
0.990   1165.000
Name: totalRatingCount, dtype: float64


In [19]:
popularity_threshold = 1000
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')

In [ ]:
combined = rating_popular_book.merge(users, left_on = 'user_id', right_on = 'user_id', how = 'left')

#user_rating = combined.drop_duplicates(['user_id', 'title'])
combined.head()

In [ ]:
user_rating_pivot2 = combined.pivot(index = 'user_id', columns = 'title', values = 'rating').fillna(0)
user_rating_pivot2.head()

In [ ]:
#Trasponemos esta matriz para que bookTitles se hagan Rows 
#y los Users sean las columnas
user_rating_pivot2.shape

In [ ]:
X = user_rating_pivot2.values.T
X.shape

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

In [ ]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

In [ ]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

def df_recommend(recommend):
    recommend=recommend[:5]
    year=[]
    author = []
    image_url=[]
    for i in recommend:
        for j in combined.index:
            if combined['title'][j]==i:
                year.append(combined['publication_year'][j])
                #author.append(combined['bookAuthor'][j])
                image_url.append(combined['image_url'][j])  
                break
    recommend_df=pd.DataFrame([recommend, year, image_url]).T
    recommend_df.columns=['Recommend Books','Year of Publication','Image']
    
    image_cols = ['Image']


    format_dict = {}
    for image_col in image_cols:
        format_dict[image_col] = path_to_image_html

    display(HTML(recommend_df[0:10].to_html(escape=False ,formatters=format_dict)))

In [ ]:
def recommend_book(df_pivot, corr_mat):
    name=input("Enter the Name of the Book : ")
    book_names = user_rating_pivot2.columns
    book_list = list(book_names)
    try:
        if name in book_list:
            book_index = book_list.index(name) 
            corr_book = corr[book_index] 
            print("Recommendation for " + name + " are:")
            recommend=list(book_names[(corr_book<1.0) & (corr_book>0.8)][1:])
            df_recommend(recommend)

        else:
            name=" "+name
            book_index = book_list.index(name) 
            corr_book = corr[book_index] 
            print("Recommendation for " + name + " are:")
            recommend=list(book_names[(corr_book<1.0) & (corr_book>0.8)][1:])
            df_recommend(recommend)
    except:
        print("Enter the Book Name Again")
        recommend_book(df_pivot,corr_mat)

In [ ]:
recommend_book(user_rating_pivot2, corr)